In [11]:
#注：由于简化问题，此系统无法选择连上三节课的课程（大部分课程为一次两节课）
#且不会出现选了连着3节课报错的问题，直接根据course_credits进行控制课程检索，根本就添加不了连着上3节的课

from tkinter import *
import tkinter as tk  
import tkinter.ttk
from tkinter import ttk
import tkinter.messagebox
import pickle
from PIL import Image,ImageTk
from tkinter import filedialog
import numpy as np
import pandas as pd
import collections

#1. 用户登录界面
def usr_login():    # 定义用户登录功能

    #获取用户输入的usr_name和usr_pwd
    usr_name = var_usr_name.get()
    usr_pwd = var_usr_pwd.get()

    # 设置异常捕获，当第一次访问用户信息文件时是不存在的，设置异常捕获
    # 使程序将输入的信息和文件中的信息匹配
    try:
        with open('users_info.pickle', 'rb') as usr_file:
            usrs_info = pickle.load(usr_file)
    except FileNotFoundError:
        # 没有读取到'usr_file'的时候，程序会创建'usr_file'这个文件
        # 并将管理员的用户和密码写入，即初始用户名为'admin'密码为'admin'
        with open('users_info.pickle', 'wb') as usr_file:
            usrs_info = {'admin': 'admin'}
            pickle.dump(usrs_info, usr_file)
            usr_file.close()    # 必须先关闭，否则pickle.load()会出现EOFError: Ran out of input
            
    # 如果用户名和密码与文件中的匹配成功，则登录成功
    if usr_name in usrs_info.keys():
        if usr_pwd in usrs_info.values():
            tkinter.messagebox.askokcancel('欢迎👏','准备好进行智能排课了吗 %s同学'%usr_name)
            window.destroy()
            load_data_page()
            
            
       # 如果用户名匹配成功，而密码输入错误，则会弹出'密码错误，请重新输入'
        else:
            tkinter.messagebox.showerror(title='出错了!', message ="密码错误，请重新输入")
    # 如果发现用户名不存在，提示需不需要注册新用户
    else:  
        is_sign_up = tkinter.messagebox.askyesno('欢迎👏','您尚未注册，请问需要进行注册吗？')
        if is_sign_up:
            usr_sign_up()

    
# 定义用户注册功能
#怎么通过messagebox的按钮链接到下一个下一个窗口？
def usr_sign_up():  #判断是否注册成功

    def sign_to_python_Website():   # 获取注册时所输入的信息，并做基本判断

        #获取用户注册时输入的新用户名、第一次密码和第二次密码
        new_name = var_new_name.get()
        new_pwd = var_new_pwd.get()
        new_pwd_confirm=var_new_pwd_confirm.get()

        # 打开记录数据的文件，将注册信息读出
        with open('users_info.pickle', 'rb') as usr_file:
            usrs_info = pickle.load(usr_file)

        # 判断，如果两次密码输入不一致，则提示'Error,Password and confirm password must be the same!'
        if new_pwd != new_pwd_confirm:
            tkinter.messagebox.showerror(title='出错了!', message = '验证密码错误，请重新输入')

        # 如果用户名已经在我们的数据文件中，则提示'Error, The user has already signed up!'
        elif new_name in usrs_info.keys():
            tkinter.messagebox.showerror(title='出错了!', message ='您已注册过账户，请直接登录')
        
        # 最后如果输入无以上错误，则将注册输入的信息记录到文件当中，并提示注册成功Welcome！,You have successfully signed up!，然后销毁窗口。--请自行补充
        else:
            tkinter.messagebox.askyesno('欢迎👏','您已成功注册账户！')
            with open('users_info.pickle','wb') as usr_file:
                add_info={new_name:new_pwd}
                pickle.dump(add_info, usr_file)
                usr_file.close()
        # 销毁窗口。
        window_sign_up.destroy()

    # 定义长在sign_up注册窗口的形状
    
    window_sign_up = tk.Toplevel(window)
    window_sign_up.geometry('300x200')
    window_sign_up.title('注册窗口')

    var_new_name = tk.StringVar()   # 将输入的注册名赋值给变量
    var_new_pwd = tk.StringVar()    # 将第一次输入的密码赋值给变量
    var_new_pwd_confirm = tk.StringVar()    # 将第二次输入的密码赋值给变量
    var_new_name.set('yolanda@shu.com')  # 将最初显示定为'example@shu.com'

    tk.Label(window_sign_up, text='用户名: ').place(x=10, y=10)  # 将'User name:'放置在坐标（10,10）。

    entry_new_name = tk.Entry(window_sign_up, textvariable=var_new_name)  # 创建一个注册名的'entry'，变量为new_name
    entry_new_name.place(x=130, y=10)  # 'entry'置于坐标（150,10）

    tk.Label(window_sign_up, text='密码:').place(x=10, y=50)
    tk.Label(window_sign_up, text='验证密码:').place(x=10, y=90)
 
    entry_usr_pwd1 = tk.Entry(window_sign_up, show='*',textvariable=var_new_pwd).place(x=130, y=50)
    entry_usr_pwd2 = tk.Entry(window_sign_up, show='*',textvariable=var_new_pwd_confirm).place(x=130, y=90)

    # 按钮点击后，执行sign_to_Hongwei_Website
    btn_comfirm_sign_up = tk.Button(window_sign_up, text='注册', command=sign_to_python_Website)
    btn_comfirm_sign_up.place(x=180, y=120)


def load_data_page():        #传入课程信息文件
    
    def openFile():
        global filepath
        filepath = entryFile.get()
        if filepath == "":
            filepath = filedialog.askopenfilename()
        else:
            filepath = filedialog.askopenfilename(initialdir=filepath)
        var.set(filepath)
        #entryFile.delete(0, tk.END)
        #entryFile.insert(0, filepath)
        #file_dir_divider(filepath)                                          #这个似乎没什么用吗

    def read_file(path):
        data_arr = pd.read_csv(path, index_col = '课程号')
        return data_arr
        
    def load_data(): 
        global data
        try:
            data = read_file(filepath)
            switch = True
        except Exception:
            tkinter.messagebox.showerror(title='出错了!', message ='请获取csv格式的课程文件')      #只能获取csv文件
            switch = False
        if switch:
            #display(data)
            clean_data()
            root1.destroy()
            info_page()
        
    
    def clean_data():
        global data, course_concat, course_credits
        #数据预处理部分
        course_data = data[['课程名称','学分','上课时间']].copy()

        #替换关键字
        def keyword_replace(course_data):
            old = ['男', '女','单', '双', '上','\xa0']
            new = [' ']*len(old)
            for i in range(len(old)):
                course_data['上课时间'] = course_data['上课时间'].str.replace(old[i],new[i],5)
            return course_data
        course_data = keyword_replace(course_data)
        course_split = course_data['上课时间'].str.split(' ', expand = True)

        #生成非常重要的时间列表！
        week = ['一','二','三','四','五']                            
        periods = ['1-2','3-4','5-6','7-8','9-10','11-12']
        time_list = []  
        for i in range (len(week)):
            for j in range(len(periods)):
                time_list.append(week[i]+periods[j])
        #生成time_list_alt将体育课01-02类似数据replace掉
        week_alt = ['一','二','三','四','五']                        
        periods_alt = ['01-02','03-04','05-06','07-08','09-10','11-12']
        time_list_alt = []  
        for i in range (len(week_alt)):
            for j in range(len(periods_alt)):
                time_list_alt.append(week_alt[i]+periods_alt[j])

        #替换过程
        for i in range(len(time_list_alt)):
            course_split[0] = course_split[0].str.replace(time_list_alt[i],time_list[i],1)

        #将其余全部设为空值
        for i in range (course_split.shape[1]):            #course_split.shape[1]为总列数
            for j in range(course_split.shape[0]):         #course_split.shape[0]总行数
                if course_split[i][j] not in time_list:
                    course_split[i][j] = np.NaN
        course_split = course_split.dropna(axis=1,how='all') #删除全部为NaN的列    
        course_credits = pd.concat([course_data.drop('上课时间', axis=1), course_split],axis=1)
        #生成column_list课程时间列名组成的表
        column_list = []
        for i in range(course_credits.shape[1]-2):                 #减2是为了去掉"课程名称"和"学分"列
            column_list.append(i)
        column_list
        #删除所有课程时间为NaN的课程行，学一学，删除指定多列同时为NaN的行
        course_credits = course_credits.dropna(subset=column_list, how='all') 
        #删除相同的重复行
        course_credits = course_credits.drop_duplicates(keep='first')

        time_code = []                 #time_code里的值应包含两个维度[0]代表星期,[1]代表时间段
        for i in range(5):              #代表星期
            for j in range(6):          #代表时间段
                time_code.append([i,j])
        code_dict = dict(zip(time_list,time_code))

        #对课程各个时间段的数据进行编码
        for i in range(course_credits.shape[1]-2):                  #减2是为了去掉"课程名称"和"学分"列
            course_credits[i] = course_credits[i].map(code_dict);       #通过map函数编码
        #display(course_credits)
        course_concat = course_credits.drop('学分',axis=1)
    
    global filepath
    filepath = ''

    root1=tkinter.Tk()
    root1.title('获取课表信息')
    root1['height']=130
    root1['width']=320
    
    labelFile=tkinter.Label(root1,text='获取文件：',width=50)
    labelFile.place(x=10,y=25,width=60,height=30)
    var=tkinter.StringVar(value=filepath)
    entryFile=tkinter.Entry(root1,width=120,textvariable=var)
    entryFile['state'] = 'readonly'                         #只读
    entryFile.place(x=70,y=25,width=120,height=25)
    buttonFile=tkinter.Button(root1,text="浏览...",command=openFile)
    buttonFile.place(x=200,y=25,width=100,height=25)
    
    bt=tkinter.Button(root1,text="确认",width=80,command=load_data)
    bt.place(x=90,y=65,width=150,height=50)
    

    

    

    

def info_page():  
    def addInformation():
        global total_credits,total_credits_temp,course_select
        try:           
            code = list(set(course_credits[course_credits['课程名称'] == varName.get()].index.tolist()))[0]
            code_alt = varClass.get()
            class_credit = int(np.unique(course_credits.loc[[varClass.get()],['学分']])) 
            if code == code_alt:                              #判断课程名与课程号信息匹配后添加课程
                total_credits_temp += class_credit   #加学分
                if total_credits_temp<=35 and code not in course_select:                         
                    #加入course_select列表
                    course_select.append(code)
                    #设置总学分
                    total_credits = total_credits_temp
                    varSum.set(total_credits)
                    #显示所选科目
                    result='课程名称:'+entryName.get()
                    result=result+';课程号:'+entryClass.get()
                    listboxStudent.insert(0,result)
                else:
                    if code in course_select:
                        tkinter.messagebox.showerror(title='出错了!', message ='你也太卷了, 一门课还要修两次')        
                    else:
                        tkinter.messagebox.showerror(title='出错了!', message ='你也太卷了, 总学分会超过35分的')
                    total_credits_temp -= class_credit
            else:
                tkinter.messagebox.showerror(title='出错了!', message ='瞪大眼再匹配下课程名与课程号')
                
        except Exception as e:
            #print(e)
            size = np.unique(course_credits[course_credits['课程名称'] == varName.get()].index.tolist()).size         #判断是否存在该课程
            if size>0:
                tkinter.messagebox.showerror(title='出错了!', message = '瞪大眼再匹配下课程名与课程号')     
            else:
                tkinter.messagebox.showerror(title='出错了!', message = '这门课不翼而飞了哟')            

    def deleteSelection():
        global total_credits,total_credits_temp,course_select
        selection=listboxStudent.curselection()
        if not selection:
            tkinter.messagebox.showinfo(title='Information',message='No Selection')
        else:
            #print(selection[0])                                           #‘#’编写时测试用的代码
            listboxStudent.delete(selection)
            
            code_position = len(course_select)-1-selection[0]    #定位在list中的位置，与listbox里的位置是相逆的
            
            #删学分
            total_credits_temp -= int(np.unique(course_credits.loc[[course_select[code_position]],['学分']]))
            total_credits = total_credits_temp
            varSum.set(total_credits)
            #print(total_credits_temp)
            
            #删list中的课程号
            course_select.pop(code_position)           #注！ 这里删除的方式是逆着删的
            #print(course_select)

    def submitFile():                                   #这边加入我数据处理的代码
        try:
            course_combination()
            if CheckVar.get() == 1:
                no_morning_8am()
            if len(final_tables)==0:
                tkinter.messagebox.showerror(title='出错了!', message='有课程冲突啦，排不出课表')
            else:
                Schedule_page()

            
            #for i in range(len(final_tables)):
                #display(pd.DataFrame(final_tables[i]))
            
        
        except Exception:
            tkinter.messagebox.showerror(title='出错了!', message='空空的一片，啥也没选??')
        
    def Page_Return():
        root.destroy()
        load_data_page()
    
    #检索课程对应课程号
    def Class_getcode():
        getcode = np.unique(course_credits[course_credits['课程名称'] == varName.get()].index.tolist())
        if getcode.size > 0:                                                            #注意学一下，判断一个array存不存在值
            tkinter.messagebox.showinfo(title='提示', message='%s对应课程号为%s'\
                            %(varName.get(),getcode))
            varClass.set(getcode[0])
        else:
            tkinter.messagebox.showerror(title='出错了!', message ='嗯？？没有这门课吧?')
    #检索课程号对应课程名                                    
    def Class_getname():
        try:
            getName = np.unique(course_credits.loc[[varClass.get()],['课程名称']])
            tkinter.messagebox.showinfo(title='提示', message='%s对应课程名为%s'\
                            %(varClass.get(),getName))
            varName.set(getName[0])
        except Exception:
            tkinter.messagebox.showerror(title='出错了!', message ='嗯？？没有这个课程号吧?') 
      
    
    
    def no_morning_8am():                    #获得早八最少的课表
        global final_tables
        no_8am_tables=[]
        num = []
        for i in range(len(final_tables)):
            morning_8am = final_tables[i][0]
            counter = collections.Counter(morning_8am)
            num.append(counter.get('0', 0))                 #获取每张课表无早八的天数
        indices=[i for i, x in enumerate(num) if x==max(num)]       #求出所有最少早八课表的索引
        for i in indices:
            no_8am_tables.append(final_tables[i])
        #print(len(no_8am_tables))
        final_tables = no_8am_tables
        
    
    def course_combination():
        global course_select, course_concat, final_tables

        #根据检索生成信息表（三维）
        course_info = []          
        for i in course_select:
            course_info.append(course_concat.loc[[i]].values)
        course_info = sorted(course_info, key = lambda x: len(x))  #对列表进行排序

        
        #实现组合排课机制，核心！！！（与课堂演示版有所更改，进行了简化）
        def table_generator(course_info):
            global init_array, count_courses, n, total_combos   
            
            #每一门课形成一张array
            course_array=[[] for i in range(len(course_info))]         #创建一个二维列表，学习一下 
            for i in range(len(course_info)):
                for j in range(len(course_info[i])):
                    temp_array = np.array([[[0,0] for j in range(6)] for i in range(5)]) #生成空array课表,学习一下生成的这个表的形式
                    for k in course_info[i][j]:
                        if isinstance(k,list):
                            temp_array[k[0]][k[1]]=[i,1]         #赋一个列表，索引[0]代表课程名，[1]默认为1为后续进行判断是否有叠加
                                                                     #例如用[0,1]表示数据库基础，用[3,1]表示经济学(1)
                    course_array[i].append(temp_array)                      


            #通过递归的方式进行嵌套for循环来组合所有array进行相加           
            init_array = np.array([[[0,0] for j in range(6)] for i in range(5)])               
            count_courses = len(course_array)
            n = 0
            total_combos = []
            def array_combo():
                global count_courses, n, init_array
                if n < count_courses-1:
                    for i in course_array[n]:
                        init_array+=i              #进行各课程数组的叠加
                        n+=1
                        array_combo()
                        n-=1
                        init_array-=i
                else:
                    for j in course_array[n]:
                        init_array+=j
                        total_combos.append(init_array.copy().tolist())                  #得copy一下，转换成列表方便后续进行删除
                        init_array-=j

            array_combo()


            overlapped_arrays=[]             #保存所有存在冲突的组合
            flag = False
            for i in total_combos:
                for j in range(len(i)):
                    for k in range(len(i[j])):
                        if i[j][k][1]>1:               #对每一个组合进行判断是否存在叠加，即是否存在课程冲突
                            overlapped_arrays.append(i)
                            flag = True
                            break
                    if flag:
                        flag = False
                        break

            #从所有组合中去除冲突组合
            for i in overlapped_arrays:
                if i in total_combos:
                    total_combos.remove(i)       
            #每个组合转换成DataFrame
            for i in range(len(total_combos)):
                total_combos[i] = pd.DataFrame(total_combos[i])  
            #建立用于map的字典
            code_map = ['[0, 0]']            #注意这边有一个空格
            course_map = ['0']
            for i in range(len(course_info)):
                code_map.append(str([i,1]))
            for i in range(len(course_info)):
                course_map.append(course_info[i][0][0])   
            map_dict = dict(zip(code_map, course_map))

            #display(map_dict)
            for i in range(len(total_combos)):
                for j in range(total_combos[i].shape[1]):
                    total_combos[i][j] = total_combos[i][j].astype('str').map(map_dict);

            return total_combos
    
    
        final_tables = table_generator(course_info)        
        for i in range(len(final_tables)):
            final_tables[i] = np.array(final_tables[i]).T   #对final_tables里的每张课表进行转置，并且转成np.array，用于no_morning_8am
        
        
    global course_select, total_credits_temp
    
    #初始化总学分
    total_credits_temp = 0

    #初始化所选择课程的列表
    course_select=[]   
    
    #2. 输入课程信息界面
    root=tkinter.Tk()
    root.title('课程输入界面')
    root['height']=350
    root['width']=320

    labelName=tkinter.Label(root,text='课程名称：',justify=tkinter.RIGHT,width=50)
    labelName.place(x=10,y=10,width=60,height=30)
    varName=tkinter.StringVar(value='')
    entryName=tkinter.Entry(root,width=120,textvariable=varName)
    entryName.place(x=70,y=10,width=150,height=25)
    buttonName=tkinter.Button(root,text="检索课程号",command=Class_getcode)         #,command=Class_getcode    搜索课程号函数
    buttonName.place(x=225,y=11,width=80,height=20)

    labelClass=tkinter.Label(root,text='课程号：',justify=tkinter.RIGHT,width=50)
    labelClass.place(x=10,y=50,width=60,height=30)
    varClass=tkinter.StringVar(value='')
    entryClass=tkinter.Entry(root,width=120,textvariable=varClass)
    entryClass.place(x=70,y=50,width=150,height=25)
    buttonName=tkinter.Button(root,text="检索课程名",command=Class_getname)         #,command=Class_getname    搜索课程名函数
    buttonName.place(x=225,y=51,width=80,height=20)
    
    
    buttonDelete=tkinter.Button(root,text='删除',width=100,command=deleteSelection)
    buttonDelete.place(x=180,y=100,width=100,height=20)

    
    listboxStudent=tkinter.Listbox(root,width=300)                  #这里要改下， 改好了       ,
    listboxStudent.place(x=10,y=130,width=300,height=100)
    #为listbox添加scrollbar
    sc = tkinter.Scrollbar(listboxStudent)
    sc.pack(side=tkinter.RIGHT, fill=tkinter.Y)
    listboxStudent.config(yscrollcommand=sc.set)
    sc.config(command = listboxStudent.yview)                        #补充重要两个config

    labelSum=tkinter.Label(root,text='总学分：',justify=tkinter.RIGHT,width=50)
    labelSum.place(x=10,y=230,width=60,height=30)
    varSum=tkinter.StringVar(value=0)
    entrySum=tkinter.Entry(root,width=120,textvariable=varSum,state='disabled')
    entrySum.place(x=70,y=235,width=30,height=25)
    
    CheckVar=tkinter.IntVar()
    no_8am = tkinter.Checkbutton(root, text='我要跟早八说拜拜👋',variable = CheckVar)
    no_8am.place(x=120,y=235,width=200,height=25)
    
    buttonAdd=tkinter.Button(root,text='添加',width=100,command=addInformation)
    buttonAdd.place(x=50,y=100,width=100,height=20)
    
    buttonSubmit=tkinter.Button(root,text="确认",width=80,command=submitFile)
    buttonSubmit.place(x=90,y=280,width=150,height=50)
    
    buttonReturn=tkinter.Button(root,text="返回",width=10,command=Page_Return)
    buttonReturn.place(x=5, y=315, width=50, height=30)
    
    #return(submitFile)

def Schedule_page():
    
    #3. 生成课表界面
    window=tk.Tk()
    window.title("生成课表")
            
    tree=ttk.Treeview(window)
    tree.pack(side=LEFT)

    #定义列title（接受一个元组)
    tree["columns"]=('mon','tues','wed','thu','fri')                    #这里的问题？？不，没有问题吧

    #设置列宽度
    tree.column('#0', width=90, stretch=NO)                            #对Treeview中的第一列进行设置     
    tree.column('mon', width=110, stretch=NO)
    tree.column('tues', width=110, stretch=NO)
    tree.column('wed', width=110, stretch=NO)
    tree.column('thu', width=110, stretch=NO)
    tree.column('fri', width=110, stretch=NO)
    #设置表头
    tree.heading('#0',text='时间段')
    tree.heading('mon',text='周一')
    tree.heading('tues',text='周二')
    tree.heading('wed',text='周三')
    tree.heading('thu',text='周四')
    tree.heading('fri',text='周五')

    periods = ['1-2','3-4','5-6','7-8','9-10','11-12']

    for i in range(len(final_tables)):
        table = pd.DataFrame(final_tables[i])
        for j in range (table.shape[0]):
            tree.insert('','end',text='{}'.format(periods[j]),values=tuple(table.iloc[j].values))
        tree.insert('','end',text='',values='')
        if i+1!=len(final_tables):
            tree.insert('','end',text='下一张课表',values='')

    tree_sc = tkinter.Scrollbar(window, orient='vertical',command = tree.yview)
    tree_sc.pack(side=RIGHT, fill=Y)
    tree.config(yscrollcommand=tree_sc.set)




# 实例化object，建立窗口window
window = tk.Tk()
window.title('智能排课系统')   # 给窗口的可视化起名字
window.geometry('400x300')  # 设定窗口的大小(长 * 宽) 

var_usr_name=StringVar()
var_usr_pwd=StringVar()

# 设置绿色背景和图片。背景画布尺寸为：width=550, height=110                                  
# 图片为'钱伟长图书馆.JPG'
canvas = tk.Canvas(window, width=550, height=110, bg='white')
image=Image.open("钱伟长图书馆.JPG")
im=ImageTk.PhotoImage(image)
canvas.create_image(300,80,anchor='center',image=im)
canvas.pack()

#标签信息
tk.Label(window, text='欢迎来到智能排课系统',font=('Arial', 14)).pack()


# 用户信息
tk.Label(window, text='用户名:', font=('Arial', 14)).place(x=10, y=170)
tk.Label(window, text='密码:', font=('Arial', 14)).place(x=10, y=210)

# 用户登录输入框entry，位置：place(x=120,y=175)
tk.Entry(window, textvariable=var_usr_name).place(x=120, y=175)

# 密码输入框，位置：place(x=120,y=215)
tk.Entry(window, show='*', textvariable=var_usr_pwd).place(x=120, y=215)

# login and sign up 按钮-login位置place(x=120, y=240)，sign_up位置place(x=200, y=240)
login_btn = tk.Button(window, text='登录', command=usr_login).place(x=120, y=240)
sign_up_btn = tk.Button(window, text='注册', command=usr_sign_up).place(x=200, y=240)


window.mainloop()

